In [ ]:
import torch 

In [ ]:
from IPython.display import Image 

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [ ]:
torch.cuda.get_arch_list() 

In [ ]:
print('All set. Using PyTorch version %s with %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Training the yolov5 Model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data dataset.yaml --weights yolov5s.pt --cache

In [ ]:
# Progress in Google collab

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Validation in Google collab

Image(filename='runs/train/exp2/val_batch0_labels.jpg', width=1000)

# Testing the Model:

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source runs/train/exp/weights/test/images

In [ ]:
# Convert weights to fp16 TFLite model

!python export.py --weights runs/train/exp/weights/best.pt --include tflite --img 640 --data data/datas.yaml

In [ ]:
#Convert weights to int8 TFLite model

!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --int8 --img 640 --data data/datas.yaml

In [ ]:
!python detect.py --weights runs\train\exp\weights\best-fp16-metadata.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images

In [ ]:
#After running the above command, you will have best-fp16.tflite in runs\train\exp2\weights folder

# Lets run the created tflite model.

!python detect.py --weights runs\train\exp\weights\best-fp16.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images --data data/datas.yaml

In [ ]:
%cd yolov5-tflite

In [ ]:
!python metadata_writer_v1.py --model_file best-fp16.tflite --label_file labels.txt

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

tf.__version__

In [ ]:
#Quantization 

#dynamic
saved_model_dir = "runs/train/exp/weights/best_saved_model"

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

In [ ]:
#integer with float fallback

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 640, 640, 3)
      yield [data.astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
open("best-fp16 Integer float fallback quantization.tflite", "wb").write(tflite_quant_model)

In [ ]:
#Integer float fallback quantization

import tensorflow as tf

train_images = "runs/train/exp/weights/train/images"

def representative_data_gen():
    # Assuming you are loading image files from the specified directory
    image_files = tf.data.Dataset.list_files(train_images + '/*.jpg')  # Update file extension if needed
    for img_path in image_files.take(100):  # Take 100 images for representative dataset
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)  # Assuming RGB images
        img = tf.image.resize(img, [640, 640])  # Resize if needed
        img = tf.expand_dims(img, 0)  # Add batch dimension
        img = img / 255.0  # Normalize to [0, 1]
        yield [img]

# Assuming saved_model_dir is the directory where your Keras model is saved
saved_model_dir = "runs/train/exp/weights/best_saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()
open("best-fp16 Integer float fallback quantization 2.tflite", "wb").write(tflite_model_quant)

In [ ]:
#Float 16

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

open("best-fp16 Integer float16.tflite", "wb").write(tflite_model_quant)

In [ ]:
!python detect.py --weights runs\train\exp\weights\best-fp16-Dynamic-range-quantization.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images --data data/datas.yaml

In [ ]:
!python detect.py --weights runs\train\exp\weights\best-fp16-Integer-float-fallback-quantization-1-random-dataset.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images --data data/datas.yaml

In [ ]:
!python detect.py --weights runs\train\exp\weights\best-fp16-Integer-float-fallback-quantization-2-train-dataset.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images --data data/datas.yaml

In [ ]:
!python detect.py --weights runs\train\exp\weights\best-fp16-float16.tflite --img 640 --conf 0.25 --source runs/train/exp/weights/test/images --data data/datas.yaml